# A Fast, Lightweight nnUNet-based Brain Tumor Segmentation Model


In [1]:
# Install nnU-Net v2 and dependencies
!pip install -q nnunetv2 SimpleITK nibabel matplotlib tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 kB 4.9 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.8 MB/s eta 0:00:00
   ━━

In [2]:
!gdown -q 1kI63BB_jc1C7kfPdubKr6cDAqyb9StYQ   # Brats_2025 Dataset
!unzip -q ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData.zip    #Training Dataset

In [ ]:
import os

os.environ["nnUNet_raw"] = "/kaggle/working/nnUNet_raw"
os.environ["nnUNet_preprocessed"] = "/kaggle/working/nnUNet_preprocessed"
os.environ["nnUNet_results"] = "/kaggle/working/nnUNet_results"

# Confirm they're set
!echo $nnUNet_raw
!echo $nnUNet_preprocessed
!echo $nnUNet_results

In [ ]:
import os, shutil, glob, json

# === User: Update path to your attached dataset ===
DATASET_INPUT_PATH = "/kaggle/working/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData_V2"  # <-- change this

# nnU-Net directory structure
RAW_DIR = "nnUNet_raw"
PREPROC_DIR = "nnUNet_preprocessed"
RESULTS_DIR = "nnUNet_results"
DATASET_ID = 2023
DATASET_NAME = f"Dataset{DATASET_ID}"

imagesTr = os.path.join(RAW_DIR, DATASET_NAME, "imagesTr")
labelsTr = os.path.join(RAW_DIR, DATASET_NAME, "labelsTr")
os.makedirs(imagesTr, exist_ok=True)
os.makedirs(labelsTr, exist_ok=True)

print('Folders ready:', imagesTr, labelsTr, sep='\n')

In [ ]:
# Updated modality mapping based on your file suffixes
modalities_map = {
    "-t1n.nii.gz": "0000",
    "-t1c.nii.gz": "0001",
    "-t2f.nii.gz": "0002",
    "-t2w.nii.gz": "0003"
}

# Go into each case folder and process files
case_dirs = os.listdir(DATASET_INPUT_PATH)
for case_dir in case_dirs:
    case_path = os.path.join(DATASET_INPUT_PATH, case_dir)
    if not os.path.isdir(case_path):
        continue

    for file_name in os.listdir(case_path):
        file_path = os.path.join(case_path, file_name)

        # Extract case ID (everything before the last dash)
        case_id = "-".join(file_name.split("-")[:3])

        if file_name.endswith("-seg.nii.gz"):
            shutil.copy(file_path, os.path.join(labelsTr, f"{case_id}.nii.gz"))
        else:
            for suf, ch in modalities_map.items():
                if file_name.endswith(suf):
                    shutil.copy(file_path, os.path.join(imagesTr, f"{case_id}_{ch}.nii.gz"))
                    break

# Final summary
print("Copied", len(os.listdir(imagesTr)), "images and", len(os.listdir(labelsTr)), "labels.")

In [ ]:
# Create dataset.json
dataset_json = {
    "channel_names": {str(i): m.strip("_") for i, m in enumerate(['t1n','t1c','t2f','t2w'])},
    "labels": {"background": 0, "WT": 1, "ET": 2, "TC": 3},
    "numTraining": len(os.listdir(labelsTr)),
    "file_ending": ".nii.gz",
    "reference": "Created via starter notebook"
}
with open(os.path.join(RAW_DIR, DATASET_NAME, "dataset.json"), "w") as fp:
    json.dump(dataset_json, fp, indent=4)
print("Saved dataset.json")

In [ ]:
# Preprocess (takes time)
!nnUNetv2_plan_and_preprocess -d 2023 -pl ResEncUNetPlanner --verify_dataset_integrity

In [ ]:
#Training with 100 epochs
!nnUNetv2_train 2023 3d_fullres 0 -p nnUNetResEncUNetPlans -tr nnUNetTrainer_100epochs

## Validation/Inference

In [ ]:
!gdown -q 1GnGsfUb__HUfRcbMvfFGeJ94DdEit0gW
!unzip -q BraTS2024-SSA-Challenge-ValidationData.zip

In [ ]:
VALIDATION_IMAGES_DIR = "/kaggle/working/BraTS2024-SSA-Challenge-ValidationData"
OUTPUT_PRED_DIR = "/kaggle/working/validation_predictions"
os.makedirs(OUTPUT_PRED_DIR, exist_ok=True)

In [ ]:
import os
import shutil

base_path = '/kaggle/working/BraTS2024-SSA-Challenge-ValidationData'
output_path = '/kaggle/working/nnUNet_input'

os.makedirs(output_path, exist_ok=True)

for case_folder in os.listdir(base_path):
    case_path = os.path.join(base_path, case_folder)
    if not os.path.isdir(case_path):
        continue

    nii_files = sorted([f for f in os.listdir(case_path) if f.endswith('.nii.gz')])
    
    for i, f in enumerate(nii_files):
        src = os.path.join(case_path, f)
        dst = os.path.join(output_path, f"{case_folder}_{i:04}.nii.gz")
        shutil.copy(src, dst)

print("Flattened and renamed images ready in:", output_path)

In [ ]:
!nnUNetv2_predict \
  -i /kaggle/working/nnUNet_input \
  -o /kaggle/working/validation_predictions \
  -d 2023 \
  -c 20epochs__nnUNetResEncUNetPlans__3d_fullres \
  -f 0

In [ ]:
import os

pred_dir = "/kaggle/working/validation_predictions"
for filename in os.listdir(pred_dir):
    if filename.endswith(".nii.gz"):
        old_path = os.path.join(pred_dir, filename)
        # Rename to required format if needed
        # Example: "BraTS-SSA-00045-seg.nii.gz"
        new_filename = filename.replace("_0000", "").replace(".nii.gz", "-seg.nii.gz")
        new_path = os.path.join(pred_dir, new_filename)
        os.rename(old_path, new_path)